# 🏥 Crawler Industrial de Grupos WhatsApp Médicos

Extrai grupos de WhatsApp de todos os principais agregadores de vagas médicas do Brasil.

## Sites cobertos:
1. **Escala de Plantão** - HTML simples (requests)
2. **Grupos Médicos** - HTML simples (requests)
3. **Quero Plantão** - JavaScript (Playwright)
4. **Search Plantão** - JavaScript (Playwright)
5. **Plantões Médicos Brasil** - JavaScript (Playwright)

## Requisitos:
```bash
pip install requests beautifulsoup4 pandas lxml playwright
playwright install chromium
```

In [ ]:
# Instalação (rodar apenas uma vez)
# !pip install requests beautifulsoup4 pandas lxml playwright -q
!playwright install chromium

zsh:1: command not found: playwright


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from datetime import datetime
from playwright.sync_api import sync_playwright

print("✅ Bibliotecas carregadas")

✅ Bibliotecas carregadas


## 1. Funções Auxiliares

In [2]:
def extract_whatsapp_code(url):
    """Extrai o código do convite do link do WhatsApp"""
    match = re.search(r'chat\.whatsapp\.com/([A-Za-z0-9]+)', url)
    return match.group(1) if match else None

def extract_telegram_code(url):
    """Extrai o código/username do link do Telegram"""
    match = re.search(r't\.me/(?:joinchat/)?([A-Za-z0-9_-]+)', url)
    return match.group(1) if match else None

def detect_state(text):
    """Detecta o estado brasileiro no texto"""
    states = {
        'AC': ['acre'], 'AL': ['alagoas'], 'AP': ['amapá', 'amapa'],
        'AM': ['amazonas'], 'BA': ['bahia'], 'CE': ['ceará', 'ceara'],
        'DF': ['distrito federal', 'brasília', 'brasilia', ' df '],
        'ES': ['espírito santo', 'espirito santo'],
        'GO': ['goiás', 'goias'], 'MA': ['maranhão', 'maranhao'],
        'MT': ['mato grosso'], 'MS': ['mato grosso do sul'],
        'MG': ['minas gerais'], 'PA': ['pará'],
        'PB': ['paraíba', 'paraiba'], 'PR': ['paraná', 'parana'],
        'PE': ['pernambuco'], 'PI': ['piauí', 'piaui'],
        'RJ': ['rio de janeiro'], 'RN': ['rio grande do norte'],
        'RS': ['rio grande do sul'], 'RO': ['rondônia', 'rondonia'],
        'RR': ['roraima'], 'SC': ['santa catarina'],
        'SP': ['são paulo', 'sao paulo', ' sp ', 'interior-sp', 'interior sp'],
        'SE': ['sergipe'], 'TO': ['tocantins'],
        'NORTE': ['região norte', 'regiao norte'],
        'NORDESTE': ['região nordeste', 'regiao nordeste', 'nordeste'],
        'SUDESTE': ['região sudeste', 'regiao sudeste', 'sudeste'],
        'SUL': ['região sul', 'regiao sul'],
        'CENTRO_OESTE': ['centro-oeste', 'centro oeste']
    }
    
    text_lower = ' ' + text.lower() + ' '
    for state_code, patterns in states.items():
        for pattern in patterns:
            if pattern in text_lower:
                return state_code
    return 'BR'

def detect_category(text):
    """Detecta a categoria/especialidade"""
    categories = {
        'plantao_geral': ['plantão', 'plantao', 'vagas', 'emprego', 'escala'],
        'cardiologia': ['cardiologia', 'cardiologista', 'cardio'],
        'pediatria': ['pediatria', 'pediatra', 'pediátrico', 'neonat', 'neo '],
        'ginecologia': ['ginecologia', 'obstetrícia', 'obstetricia', 'gineco'],
        'ortopedia': ['ortopedia', 'ortopedista'],
        'neurologia': ['neurologia', 'neurologista', 'neuro'],
        'psiquiatria': ['psiquiatria', 'psiquiatra'],
        'anestesiologia': ['anestesiologia', 'anestesista', 'anestesia'],
        'cirurgia': ['cirurgia', 'cirurgião', 'cirurgiao'],
        'dermatologia': ['dermatologia', 'dermatologista'],
        'oftalmologia': ['oftalmologia', 'oftalmologista', 'oftalmo'],
        'urologia': ['urologia', 'urologista'],
        'emergencia': ['emergência', 'emergencia', 'urgência', 'urgencia', 'uti', 'intensiva'],
        'oncologia': ['oncologia', 'oncologista'],
        'gastroenterologia': ['gastroenterologia', 'gastro'],
        'pneumologia': ['pneumologia', 'pneumologista'],
        'infectologia': ['infectologia', 'infectologista'],
        'nefrologia': ['nefrologia', 'nefrologista'],
        'endocrinologia': ['endocrinologia', 'endocrinologista'],
        'reumatologia': ['reumatologia', 'reumatologista'],
        'geriatria': ['geriatria', 'geriatra'],
        'otorrino': ['otorrino', 'otorrinolaringologia'],
        'radiologia': ['radiologia', 'radiologista', 'ultrassonografia', 'ecografia'],
        'medicina_trabalho': ['trabalho', 'ocupacional'],
        'telemedicina': ['telemedicina', 'teleconsulta'],
        'medicina_familia': ['família', 'familia', 'comunidade'],
        'enfermagem': ['enfermagem', 'enfermeiro', 'enfermeira'],
        'odontologia': ['odontologia', 'dentista', 'odonto'],
        'fisioterapia': ['fisioterapia', 'fisioterapeuta'],
        'nutricao': ['nutrição', 'nutricao', 'nutricionista', 'nutrologia'],
        'psicologia': ['psicologia', 'psicólogo', 'psicologa'],
        'fonoaudiologia': ['fonoaudiologia', 'fonoaudiólogo'],
        'material_estudo': ['material', 'resumo', 'livro', 'artigo', 'pdf'],
        'discussao_casos': ['discussão', 'casos clínicos', 'ecg', 'eco'],
        'gestao': ['gestão', 'gestao', 'empreendedorismo'],
        'concurso_residencia': ['concurso', 'residência', 'residencia']
    }
    
    text_lower = text.lower()
    for category, patterns in categories.items():
        for pattern in patterns:
            if pattern in text_lower:
                return category
    return 'medicina_geral'

print("✅ Funções auxiliares carregadas")

✅ Funções auxiliares carregadas


## 2. Crawler com Requests (sites HTML simples)

In [3]:
def crawl_with_requests(url, source_name):
    """Crawler para sites que não usam JavaScript"""
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
    except Exception as e:
        print(f"❌ Erro ao acessar {url}: {e}")
        return []
    
    soup = BeautifulSoup(response.text, 'lxml')
    groups = []
    
    # WhatsApp
    for link in soup.find_all('a', href=re.compile(r'chat\.whatsapp\.com')):
        href = link.get('href')
        name = link.get_text(strip=True) or 'Sem nome'
        
        # Tenta pegar contexto do elemento pai
        parent = link.find_parent(['div', 'section', 'article', 'li'])
        if parent:
            title = parent.find(['h2', 'h3', 'h4', 'h5', 'strong', 'b'])
            if title:
                name = title.get_text(strip=True)
        
        invite_code = extract_whatsapp_code(href)
        if invite_code and len(name) > 2:
            groups.append({
                'source': source_name,
                'platform': 'whatsapp',
                'name': name[:100],  # limita tamanho
                'url': href,
                'invite_code': invite_code,
                'state': detect_state(name),
                'category': detect_category(name),
                'scraped_at': datetime.now().isoformat()
            })
    
    # Telegram
    for link in soup.find_all('a', href=re.compile(r't\.me')):
        href = link.get('href')
        name = link.get_text(strip=True) or 'Sem nome'
        
        invite_code = extract_telegram_code(href)
        if invite_code and len(name) > 2:
            groups.append({
                'source': source_name,
                'platform': 'telegram',
                'name': name[:100],
                'url': href,
                'invite_code': invite_code,
                'state': detect_state(name),
                'category': detect_category(name),
                'scraped_at': datetime.now().isoformat()
            })
    
    return groups

print("✅ Crawler requests carregado")

✅ Crawler requests carregado


## 3. Crawler com Playwright (sites JavaScript)

In [4]:
def crawl_with_playwright(url, source_name, wait_time=5000, scroll=True):
    """Crawler para sites que carregam conteúdo via JavaScript"""
    
    groups = []
    
    with sync_playwright() as p:
        # Lança navegador headless
        browser = p.chromium.launch(headless=True)
        context = browser.new_context(
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        )
        page = context.new_page()
        
        try:
            print(f"   Acessando {url}...")
            page.goto(url, timeout=60000)
            
            # Espera inicial
            page.wait_for_timeout(wait_time)
            
            # Scroll para carregar conteúdo lazy-loaded
            if scroll:
                for _ in range(5):
                    page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
                    page.wait_for_timeout(1000)
            
            # Tenta clicar em tabs/abas se existirem (comum em sites de grupos)
            try:
                tabs = page.query_selector_all('[role="tab"], .tab, .nav-link, button[data-toggle]')
                for tab in tabs[:10]:  # limita a 10 tabs
                    try:
                        tab.click()
                        page.wait_for_timeout(1500)
                    except:
                        pass
            except:
                pass
            
            # Extrai links WhatsApp
            whatsapp_links = page.query_selector_all('a[href*="chat.whatsapp.com"]')
            print(f"   Encontrados {len(whatsapp_links)} links WhatsApp")
            
            for link in whatsapp_links:
                try:
                    href = link.get_attribute('href')
                    name = link.inner_text().strip() or 'Sem nome'
                    
                    # Tenta pegar contexto
                    try:
                        parent = link.evaluate('el => el.closest("div, section, article, li")')
                        if parent:
                            # Isso é mais complexo, vamos simplificar
                            pass
                    except:
                        pass
                    
                    invite_code = extract_whatsapp_code(href)
                    if invite_code:
                        groups.append({
                            'source': source_name,
                            'platform': 'whatsapp',
                            'name': name[:100] if len(name) > 3 else f'{source_name} - Grupo',
                            'url': href,
                            'invite_code': invite_code,
                            'state': detect_state(name),
                            'category': detect_category(name),
                            'scraped_at': datetime.now().isoformat()
                        })
                except Exception as e:
                    continue
            
            # Extrai links Telegram
            telegram_links = page.query_selector_all('a[href*="t.me"]')
            print(f"   Encontrados {len(telegram_links)} links Telegram")
            
            for link in telegram_links:
                try:
                    href = link.get_attribute('href')
                    name = link.inner_text().strip() or 'Sem nome'
                    
                    invite_code = extract_telegram_code(href)
                    if invite_code:
                        groups.append({
                            'source': source_name,
                            'platform': 'telegram',
                            'name': name[:100] if len(name) > 3 else f'{source_name} - Grupo',
                            'url': href,
                            'invite_code': invite_code,
                            'state': detect_state(name),
                            'category': detect_category(name),
                            'scraped_at': datetime.now().isoformat()
                        })
                except:
                    continue
                    
        except Exception as e:
            print(f"   ❌ Erro: {e}")
        finally:
            browser.close()
    
    return groups

print("✅ Crawler Playwright carregado")

✅ Crawler Playwright carregado


## 4. Crawler específico para Quero Plantão

In [5]:
def crawl_quero_plantao():
    """Crawler específico para queroplantao.com.br/grupos
    
    Este site tem tabs por estado que precisam ser clicadas.
    """
    
    groups = []
    url = 'https://queroplantao.com.br/grupos/'
    
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        context = browser.new_context(
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        )
        page = context.new_page()
        
        try:
            print("   Acessando Quero Plantão...")
            page.goto(url, timeout=60000)
            page.wait_for_timeout(5000)
            
            # Fecha popup se existir
            try:
                page.click('text=Aceito', timeout=3000)
            except:
                pass
            try:
                page.click('[class*="close"]', timeout=2000)
            except:
                pass
            
            # Lista de estados para tentar clicar
            estados = [
                'Brasil', 'São Paulo', 'Rio de Janeiro', 'Minas Gerais', 'Bahia',
                'Paraná', 'Rio Grande do Sul', 'Santa Catarina', 'Goiás',
                'Distrito Federal', 'Ceará', 'Pernambuco', 'Pará', 'Amazonas',
                'Maranhão', 'Paraíba', 'Rio Grande do Norte', 'Alagoas', 'Sergipe',
                'Piauí', 'Espírito Santo', 'Mato Grosso', 'Mato Grosso do Sul',
                'Rondônia', 'Acre', 'Amapá', 'Roraima', 'Tocantins'
            ]
            
            # Scroll inicial
            for _ in range(3):
                page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
                page.wait_for_timeout(1000)
            
            # Tenta clicar em cada aba de estado
            for estado in estados:
                try:
                    # Tenta diferentes seletores
                    selectors = [
                        f'text="{estado}"',
                        f'button:has-text("{estado}")',
                        f'a:has-text("{estado}")',
                        f'[data-state="{estado}"]',
                        f'.tab:has-text("{estado}")',
                    ]
                    
                    for selector in selectors:
                        try:
                            element = page.query_selector(selector)
                            if element:
                                element.click()
                                page.wait_for_timeout(1500)
                                break
                        except:
                            continue
                except:
                    continue
            
            # Scroll final
            for _ in range(3):
                page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
                page.wait_for_timeout(1000)
            
            # Extrai todos os links
            whatsapp_links = page.query_selector_all('a[href*="chat.whatsapp.com"]')
            print(f"   Encontrados {len(whatsapp_links)} links WhatsApp")
            
            for link in whatsapp_links:
                try:
                    href = link.get_attribute('href')
                    name = link.inner_text().strip()
                    
                    if not name or len(name) < 3:
                        # Tenta pegar texto do elemento pai
                        try:
                            parent_text = link.evaluate('el => el.parentElement?.innerText || ""')
                            name = parent_text.strip()[:100] if parent_text else 'Quero Plantão - Grupo'
                        except:
                            name = 'Quero Plantão - Grupo'
                    
                    invite_code = extract_whatsapp_code(href)
                    if invite_code:
                        groups.append({
                            'source': 'quero_plantao',
                            'platform': 'whatsapp',
                            'name': name[:100],
                            'url': href,
                            'invite_code': invite_code,
                            'state': detect_state(name),
                            'category': detect_category(name),
                            'scraped_at': datetime.now().isoformat()
                        })
                except:
                    continue
            
            # Telegram
            telegram_links = page.query_selector_all('a[href*="t.me"]')
            print(f"   Encontrados {len(telegram_links)} links Telegram")
            
            for link in telegram_links:
                try:
                    href = link.get_attribute('href')
                    name = link.inner_text().strip() or 'Quero Plantão - Telegram'
                    
                    invite_code = extract_telegram_code(href)
                    if invite_code:
                        groups.append({
                            'source': 'quero_plantao',
                            'platform': 'telegram',
                            'name': name[:100],
                            'url': href,
                            'invite_code': invite_code,
                            'state': detect_state(name),
                            'category': detect_category(name),
                            'scraped_at': datetime.now().isoformat()
                        })
                except:
                    continue
                    
        except Exception as e:
            print(f"   ❌ Erro: {e}")
        finally:
            browser.close()
    
    return groups

print("✅ Crawler Quero Plantão carregado")

✅ Crawler Quero Plantão carregado


## 5. Executar Crawlers

In [6]:
# Lista de todos os sites para crawlear
sites = [
    # Sites com HTML simples (requests)
    {'url': 'https://escaladeplantao.com.br/grupos', 'name': 'escala_plantao', 'method': 'requests'},
    {'url': 'https://gruposmedicos.com.br/', 'name': 'grupos_medicos', 'method': 'requests'},
    
    # Sites com JavaScript (Playwright)
    {'url': 'https://queroplantao.com.br/grupos/', 'name': 'quero_plantao', 'method': 'playwright_custom'},
    {'url': 'https://web.searchplantao.com.br/grupos', 'name': 'search_plantao', 'method': 'playwright'},
    {'url': 'https://plantoesmedicosbrasil.com.br/', 'name': 'plantoes_brasil', 'method': 'playwright'},
]

all_groups = []

print("="*70)
print("   INICIANDO CRAWLER INDUSTRIAL")
print("="*70)

for site in sites:
    print(f"\n🔍 {site['name'].upper()}")
    print(f"   URL: {site['url']}")
    
    try:
        if site['method'] == 'requests':
            groups = crawl_with_requests(site['url'], site['name'])
        elif site['method'] == 'playwright_custom' and site['name'] == 'quero_plantao':
            groups = crawl_quero_plantao()
        else:
            groups = crawl_with_playwright(site['url'], site['name'])
        
        all_groups.extend(groups)
        print(f"   ✅ {len(groups)} grupos extraídos")
    except Exception as e:
        print(f"   ❌ Erro: {e}")
    
    # Intervalo entre sites
    time.sleep(2)

print(f"\n{'='*70}")
print(f"   TOTAL BRUTO: {len(all_groups)} grupos")
print("="*70)

   INICIANDO CRAWLER INDUSTRIAL

🔍 ESCALA_PLANTAO
   URL: https://escaladeplantao.com.br/grupos
   ✅ 49 grupos extraídos

🔍 GRUPOS_MEDICOS
   URL: https://gruposmedicos.com.br/
   ✅ 322 grupos extraídos

🔍 QUERO_PLANTAO
   URL: https://queroplantao.com.br/grupos/
   ❌ Erro: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

🔍 SEARCH_PLANTAO
   URL: https://web.searchplantao.com.br/grupos
   ❌ Erro: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

🔍 PLANTOES_BRASIL
   URL: https://plantoesmedicosbrasil.com.br/
   ❌ Erro: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

   TOTAL BRUTO: 371 grupos


## 6. Processar e Deduplica

In [7]:
# Cria DataFrame
df = pd.DataFrame(all_groups)

if len(df) == 0:
    print("❌ Nenhum grupo encontrado!")
else:
    # Remove duplicatas por invite_code
    antes = len(df)
    df = df.drop_duplicates(subset=['invite_code'], keep='first')
    depois = len(df)
    
    print(f"\n📊 DEDUPLICAÇÃO:")
    print(f"   Antes: {antes} grupos")
    print(f"   Depois: {depois} grupos")
    print(f"   Removidos: {antes - depois} duplicatas")


📊 DEDUPLICAÇÃO:
   Antes: 371 grupos
   Depois: 263 grupos
   Removidos: 108 duplicatas


In [8]:
# Estatísticas
if len(df) > 0:
    print("\n" + "="*70)
    print("   ESTATÍSTICAS FINAIS")
    print("="*70)
    
    print(f"\n📊 POR FONTE:")
    print(df['source'].value_counts().to_string())
    
    print(f"\n📊 POR PLATAFORMA:")
    print(df['platform'].value_counts().to_string())
    
    print(f"\n📊 POR ESTADO (top 15):")
    print(df['state'].value_counts().head(15).to_string())
    
    print(f"\n📊 POR CATEGORIA (top 15):")
    print(df['category'].value_counts().head(15).to_string())


   ESTATÍSTICAS FINAIS

📊 POR FONTE:
source
grupos_medicos    214
escala_plantao     49

📊 POR PLATAFORMA:
platform
whatsapp    263

📊 POR ESTADO (top 15):
state
BR    176
SP      7
RJ      6
DF      4
MT      4
PR      3
ES      3
TO      3
MA      3
RR      3
AP      3
AC      3
PA      3
RS      3
MG      3

📊 POR CATEGORIA (top 15):
category
medicina_geral     118
plantao_geral       49
cirurgia            11
emergencia           6
cardiologia          6
neurologia           5
radiologia           5
pediatria            5
ortopedia            4
anestesiologia       4
psiquiatria          4
infectologia         4
geriatria            3
dermatologia         3
discussao_casos      3


## 7. Filtrar Grupos Médicos

In [9]:
# Categorias que NÃO são de médicos
categorias_nao_medicas = [
    'enfermagem', 'odontologia', 'fisioterapia', 'psicologia', 
    'nutricao', 'fonoaudiologia', 'assistencia_social', 'bombeiro',
    'socorrista', 'tecnico_enfermagem', 'educacao_fisica', 'farmacia'
]

# Filtra apenas médicos
df_medicos = df[~df['category'].isin(categorias_nao_medicas)].copy()

print(f"\n🩺 GRUPOS DE MÉDICOS: {len(df_medicos)} de {len(df)} total")
print(f"\nCategorias incluídas:")
print(df_medicos['category'].value_counts().to_string())


🩺 GRUPOS DE MÉDICOS: 261 de 263 total

Categorias incluídas:
category
medicina_geral         118
plantao_geral           49
cirurgia                11
emergencia               6
cardiologia              6
neurologia               5
radiologia               5
pediatria                5
ortopedia                4
anestesiologia           4
psiquiatria              4
infectologia             4
geriatria                3
dermatologia             3
discussao_casos          3
gastroenterologia        3
endocrinologia           3
urologia                 3
reumatologia             3
pneumologia              3
nefrologia               3
material_estudo          2
otorrino                 2
medicina_familia         2
medicina_trabalho        2
oncologia                1
concurso_residencia      1
oftalmologia             1
telemedicina             1
ginecologia              1


## 8. Exportar CSVs

In [10]:
# Salva todos os grupos
df.to_csv('grupos_whatsapp_saude_COMPLETO.csv', index=False)
print(f"✅ Salvo: grupos_whatsapp_saude_COMPLETO.csv ({len(df)} grupos)")

# Salva apenas médicos
df_medicos.to_csv('grupos_whatsapp_medicos_COMPLETO.csv', index=False)
print(f"✅ Salvo: grupos_whatsapp_medicos_COMPLETO.csv ({len(df_medicos)} grupos)")

# Salva por estado (útil para operação)
estados_brasil = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 
                  'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN',
                  'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']
df_por_estado = df_medicos[df_medicos['state'].isin(estados_brasil)]
df_por_estado.to_csv('grupos_whatsapp_medicos_POR_ESTADO.csv', index=False)
print(f"✅ Salvo: grupos_whatsapp_medicos_POR_ESTADO.csv ({len(df_por_estado)} grupos)")

# Salva apenas WhatsApp (para Evolution API)
df_whatsapp = df_medicos[df_medicos['platform'] == 'whatsapp']
df_whatsapp.to_csv('grupos_whatsapp_EVOLUTION_API.csv', index=False)
print(f"✅ Salvo: grupos_whatsapp_EVOLUTION_API.csv ({len(df_whatsapp)} grupos)")

✅ Salvo: grupos_whatsapp_saude_COMPLETO.csv (263 grupos)
✅ Salvo: grupos_whatsapp_medicos_COMPLETO.csv (261 grupos)
✅ Salvo: grupos_whatsapp_medicos_POR_ESTADO.csv (80 grupos)
✅ Salvo: grupos_whatsapp_EVOLUTION_API.csv (261 grupos)


## 9. Visualizar Amostra

In [11]:
# Mostra amostra por estado
print("\n" + "="*70)
print("   AMOSTRA DE GRUPOS POR ESTADO")
print("="*70)

for state in ['SP', 'RJ', 'MG', 'BA', 'PR', 'RS', 'DF']:
    state_groups = df_medicos[df_medicos['state'] == state]
    if len(state_groups) > 0:
        print(f"\n📍 {state}: {len(state_groups)} grupos")
        for _, row in state_groups.head(3).iterrows():
            print(f"   • {row['name'][:50]}")
            print(f"     Código: {row['invite_code']}")


   AMOSTRA DE GRUPOS POR ESTADO

📍 SP: 7 grupos
   • Escala de Plantão SP  01
     Código: CIf7BTOKuqp6EUgeE4Pcjp
   • Escala de Plantão SP 02
     Código: J0Hca4lB78gJzJyocrWAUj
   • Escala de  Plantão  Interior-SP 01
     Código: LZODYjNSqwrHTtbDvkb8tg

📍 RJ: 6 grupos
   • Escala de Plantão Rio de Janeiro 01
     Código: FPKxAWoN4rMF9GthPmIwHP
   • Escala de Plantão Rio de Janeiro 02
     Código: LqCB1ySd5kxBUqAWcBPv9u
   • Escala de Plantão Rio de Janeiro 03
     Código: IBaiYgVtn6nJR8xRFYUwst

📍 MG: 3 grupos
   • Escala de Plantão Minas Gerais
     Código: IydHJ0oQ8xL0MXss1kXCrK
   • MINAS GERAIS
     Código: BCMu72FYzgl4nCQw807Z0j
   • Minas Gerais
     Código: HUjNciOWxxDLwfZmD74nFa

📍 BA: 3 grupos
   • Escala de Plantão Bahia 01
     Código: LlYuVS8yBXzCOoHWw29q7K
   • Escala de Plantão Bahia 02
     Código: Iypus185ZIwKyVyWoewWfc
   • BAHIA
     Código: FRhsQyjcTe87e5lDYF84Ml

📍 PR: 3 grupos
   • Escala de Plantão Paraná
     Código: FAgwnBaA4cD29qNt3EuGvS
   • PARANÁ
     Cód

In [12]:
# Mostra amostra por especialidade
print("\n" + "="*70)
print("   AMOSTRA DE GRUPOS POR ESPECIALIDADE")
print("="*70)

for cat in ['cardiologia', 'pediatria', 'emergencia', 'ortopedia', 'anestesiologia']:
    cat_groups = df_medicos[df_medicos['category'] == cat]
    if len(cat_groups) > 0:
        print(f"\n🏥 {cat.upper()}: {len(cat_groups)} grupos")
        for _, row in cat_groups.head(2).iterrows():
            print(f"   • {row['name'][:50]}")


   AMOSTRA DE GRUPOS POR ESPECIALIDADE

🏥 CARDIOLOGIA: 6 grupos
   • Cardiologia Pediátrica
   • Cirurgia Cardiovascular

🏥 PEDIATRIA: 5 grupos
   • CTI Pediátrico
   • Neonatologia

🏥 EMERGENCIA: 6 grupos
   • Urgência e Emergência 🚨 3
   • UTI

🏥 ORTOPEDIA: 4 grupos
   • Ortopedista
   • 🟢ORTOPEDIA 🦴

🏥 ANESTESIOLOGIA: 4 grupos
   • Anestesiologia
   • ANESTESIOLOGIA


## 10. Próximos Passos

### Para entrar nos grupos via Evolution API:

```python
import requests
import time

EVOLUTION_URL = "https://seu-evolution.com"
API_KEY = "sua-api-key"

def join_group(instance_name, invite_code):
    """Entra em um grupo de WhatsApp"""
    url = f"{EVOLUTION_URL}/group/acceptInvite/{instance_name}"
    
    response = requests.post(
        url,
        json={"inviteCode": invite_code},
        headers={"apikey": API_KEY}
    )
    return response.json()

# Lê o CSV
df = pd.read_csv('grupos_whatsapp_EVOLUTION_API.csv')

# Entra em cada grupo (com rate limiting)
for i, row in df.iterrows():
    result = join_group('julia-01', row['invite_code'])
    print(f"Grupo {row['name']}: {result}")
    
    # Máximo 5 grupos por dia por número
    if (i + 1) % 5 == 0:
        print("Limite diário atingido, trocar número ou aguardar")
        break
    
    time.sleep(60)  # 1 minuto entre cada entrada
```

### Regras de segurança:
- Máximo **5 grupos por dia** por número
- Esperar **2-3 dias** antes de começar a interagir
- Rotacionar entre múltiplos números
- Monitorar banimentos e substituir números

In [13]:
# Resumo final
print("\n" + "="*70)
print("   RESUMO FINAL")
print("="*70)
print(f"""
📊 GRUPOS EXTRAÍDOS:
   Total (todas áreas):     {len(df)} grupos
   Apenas médicos:          {len(df_medicos)} grupos
   Por estado:              {len(df_por_estado)} grupos
   WhatsApp (Evolution):    {len(df_whatsapp)} grupos

📁 ARQUIVOS GERADOS:
   • grupos_whatsapp_saude_COMPLETO.csv
   • grupos_whatsapp_medicos_COMPLETO.csv
   • grupos_whatsapp_medicos_POR_ESTADO.csv
   • grupos_whatsapp_EVOLUTION_API.csv

🚀 PRÓXIMO PASSO:
   Use o arquivo grupos_whatsapp_EVOLUTION_API.csv
   para integrar com a Evolution API e fazer a
   Júlia entrar nos grupos automaticamente.
""")


   RESUMO FINAL

📊 GRUPOS EXTRAÍDOS:
   Total (todas áreas):     263 grupos
   Apenas médicos:          261 grupos
   Por estado:              80 grupos
   WhatsApp (Evolution):    261 grupos

📁 ARQUIVOS GERADOS:
   • grupos_whatsapp_saude_COMPLETO.csv
   • grupos_whatsapp_medicos_COMPLETO.csv
   • grupos_whatsapp_medicos_POR_ESTADO.csv
   • grupos_whatsapp_EVOLUTION_API.csv

🚀 PRÓXIMO PASSO:
   Use o arquivo grupos_whatsapp_EVOLUTION_API.csv
   para integrar com a Evolution API e fazer a
   Júlia entrar nos grupos automaticamente.

